## Dependencies

In [1]:
# Dependencies to Visualize the model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float 
# from config import postgresql_pword

from flask import Flask, jsonify, render_template

In [4]:
#establish connection to SQL database in AWS - refer to https://stackoverflow.com/questions/54300263/connect-to-aws-rds-postgres-database-with-python/54313925
endpoint= 'james-bond-uwa-db.cbbzivxykkl5.ap-southeast-2.rds.amazonaws.com'
username='postgres'
password='postgres'

engine = create_engine(f'postgresql://{username}:{password}@{endpoint}:5432/James_Bond')
conn = engine.connect()

In [5]:
#extract the sql table and turn it into a dataframe
session=Session(engine)
bond_data=pd.read_sql_table("james_bond_data", conn)
session.close()

In [6]:
bond_data.head()

,year,movie,actor,director,composer,writer,cinematographer,film_location,shooting_location,bond_car,...,film_length,avg_imdb_rating,avg_rtn_tom_rating,conquest,martinis,bjb,kills_bond,kills_others,top_100_soundtrack,video_game
0,1962,Dr. No,Sean Connery,Terence Young,Monty Norman,"Richard Maibaum, Johanna Harwood & Berkely Mather",Ted Moore,"Great Britain, Jamaic","England, Jamaica",Sunbeam,...,110,7.3,7.7,3,2,1,4,8,0,0
1,1963,From Russia with Love,Sean Connery,Terence Young,John Barry,Richard Maibaum & Johanna Harwood,Ted Moore,"United Kingdom, Great Britain, Turkey, Croatia...","England, Scotland, Italy, Switzerland, Turkey",Bently,...,115,7.5,8.0,4,0,0,11,16,0,1
2,1964,Goldfinger,Sean Connery,Guy Hamilton,John Barry,Richard Maibaum & Paul Dehn,Ted Moore,"United States, Great Britain, Switzerland","England, Switzerland, United States",Aston Martin,...,110,7.8,8.4,2,1,2,9,68,1,1
3,1965,Thunderball,Sean Connery,Terence Young,John Barry,Richard Maibaum & John Hopkins,Ted Moore,"France, Great Britain, Bahamas, United States","England, France, Bahamas, United States",Aston Martin,...,130,7.0,6.8,3,0,0,20,90,1,0
4,1967,You Only Live Twice,Sean Connery,Lewis Gilbert,John Barry,Roald Dahl,Freddie Young,"United States, Russia, Kazakhstan, Norway, Japan","Japan, Spain, Norway",Toyota,...,117,6.9,6.3,3,1,0,21,175,1,0


In [7]:
bond_data.columns

Index(['year', 'movie', 'actor', 'director', 'composer', 'writer',
       'cinematographer', 'film_location', 'shooting_location', 'bond_car',
       'bond_girl_nat', 'us_grossing', 'us_gross_adj_2013',
       'us_gross_adj_2020', 'world_gross', 'world_gross_adj_2013',
       'world_gross_adj_2020', 'budget', 'budget_adj_2013', 'budget_adj_2020',
       'film_length', 'avg_imdb_rating', 'avg_rtn_tom_rating', 'conquest',
       'martinis', 'bjb', 'kills_bond', 'kills_others', 'top_100_soundtrack',
       'video_game'],
      dtype='object')

In [8]:
# Clean the dataframe by dropping irrelevant columns
bond_data=bond_data.drop(columns =['us_grossing', 'us_gross_adj_2013','world_gross', 'world_gross_adj_2013','budget', 'budget_adj_2013'])

In [9]:
# create a new column called actor id to turn actor name into numerical category
bond_data['actor id']=""
actor_list=list(bond_data['actor'].unique())
actor_id=[]
for index, row in bond_data.iterrows():
    for i in range(len(actor_list)):
        if row['actor']==actor_list[i]:
            actor_id.append(i)

bond_data['actor id']=actor_id

In [10]:
# create a new column called actor id to turn actor name into numerical category
# bond_data['actor id']=""
# bond_data['actor'].unique()
# actor_id=[]
# for index, row in bond_data.iterrows():
#     if row['actor']=="Sean Connery":
#         actor_id.append(0)
#     elif row['actor']=='George Lazenby':
#         actor_id.append(1)
#     elif row['actor']=='Roger Moore':
#         actor_id.append(2)
#     elif row['actor']=='Timothy Dalton':
#         actor_id.append(3)
#     elif row['actor']=='Pierce Brosnan':
#         actor_id.append(4)
#     elif row['actor']=='Daniel Craig':
#         actor_id.append(5)

# bond_data['actor id']=actor_id

In [11]:
bond_data.dtypes

year                      int64
movie                    object
actor                    object
director                 object
composer                 object
writer                   object
cinematographer          object
film_location            object
shooting_location        object
bond_car                 object
bond_girl_nat            object
us_gross_adj_2020       float64
world_gross_adj_2020    float64
budget_adj_2020         float64
film_length               int64
avg_imdb_rating         float64
avg_rtn_tom_rating      float64
conquest                  int64
martinis                  int64
bjb                       int64
kills_bond                int64
kills_others              int64
top_100_soundtrack        int64
video_game                int64
actor id                  int64
dtype: object

**Machine Learning**
<br>Use multiple linear regression model from SKlearn to determine the revenue for the latest Bond Movie

In [12]:
#Delete the latest Bond film from the database
training_data=bond_data[:24]
training_data[20:]

,year,movie,actor,director,composer,writer,cinematographer,film_location,shooting_location,bond_car,...,avg_imdb_rating,avg_rtn_tom_rating,conquest,martinis,bjb,kills_bond,kills_others,top_100_soundtrack,video_game,actor id
20,2006,Casino Royale,Daniel Craig,Martin Campbell,David Arnold,"Neal Purvis, Robert Wade & Paul Haggis",Phil Méheux,"Czech Republic, Pakistan, Uganda, Madagascar, ...","England, Italy, Czech republic, Bahamas, Unite...",Aston Martin,...,7.9,7.8,2,3,1,11,11,1,0,5
21,2008,Quantum of Solace,Daniel Craig,Marc Forster,David Arnold,Paul Haggis and Neal Purvis and Robert Wade,Roberto Schaefer,"Italy, England, Haiti, Austria, Bolivia, Russia","England, Italy, Spain, Panama, Mexico, Chile, ...",Aston Martin,...,6.7,6.1,1,6,0,16,15,1,1,5
22,2012,Skyfall,Daniel Craig,Sam Mendes,Thomas Newman,"Neal Purvis, Robert Wade & John Logan",Roger Deakins,"Turkey, England, China, Macau, Japan, Scotland","England, Scotland, Turkey, China",Aston Martin,...,7.8,8.2,3,1,1,26,26,1,0,5
23,2015,Spectre,Daniel Craig,Sam Mendes,Thomas Newman,"John Logan & Neal Purvis, Robert Wade & Jez Bu...",Hoyte van Hoytema,"Mexico, England, Italy, Austria, Morocco, Vati...","England, Austria, Italy, Vatican City, Mexico,...",Aston Martin,...,6.8,6.4,3,1,1,30,205,1,0,5


In [13]:
# determine columns for X variable (i.e budget, film length, user rating, etc) and y variable (expected worldwide revenue)
X=training_data[['year','budget_adj_2020','film_length', 'avg_imdb_rating','avg_rtn_tom_rating', 'kills_bond','actor id']]
y=training_data['world_gross_adj_2020'].values.reshape(-1,1)
print(X.shape, y.shape)

(24, 7) (24, 1)


In [14]:
# create train and test data from the population
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
# apply a standard scaler on the data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

C:\Users\Kevin\anaconda3\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [16]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
# X_scaled=X_scaler.transform(X)
# y_scaled=y_scaler.transform(y)

C:\Users\Kevin\anaconda3\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\Kevin\anaconda3\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [17]:
# Fit the model into the data
from sklearn import tree
model = tree.DecisionTreeRegressor()
model.fit(X_train_scaled, y_train_scaled)
training_score = model.score(X_train_scaled, y_train_scaled)
testing_score = model.score(X_test_scaled, y_test_scaled)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 1.0
Testing Score: -0.23708838689892353


In [18]:
bond_data[23:24]

,year,movie,actor,director,composer,writer,cinematographer,film_location,shooting_location,bond_car,...,avg_imdb_rating,avg_rtn_tom_rating,conquest,martinis,bjb,kills_bond,kills_others,top_100_soundtrack,video_game,actor id
23,2015,Spectre,Daniel Craig,Sam Mendes,Thomas Newman,"John Logan & Neal Purvis, Robert Wade & Jez Bu...",Hoyte van Hoytema,"Mexico, England, Italy, Austria, Morocco, Vati...","England, Austria, Italy, Vatican City, Mexico,...",Aston Martin,...,6.8,6.4,3,1,1,30,205,1,0,5


In [41]:
# extract the relevant parameter of the newest Bond movie (No Time to Die) which will be used to predict the worldwide revenue
new_movie=bond_data[0:1]
X_predict=new_movie[['year','budget_adj_2020','film_length', 'avg_imdb_rating','avg_rtn_tom_rating','kills_bond','actor id']]
X_predict_scaled=X_scaler.transform(X_predict)

C:\Users\Kevin\anaconda3\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [43]:
expected_revenue=y_scaler.inverse_transform(model.predict(X_predict_scaled).reshape(-1,1))[0][0]
expected_revenue

525785.2449

Create a new dataframe containing revenue pred to be uploaded into AWS

In [47]:
# Create blank movie list and blank predicted revenue list
movie_list=[]
predicted_revenue=[]
variables=['year','budget_adj_2020','film_length', 'avg_imdb_rating','avg_rtn_tom_rating','kills_bond','actor id']

for index, row in bond_data.iterrows():
    # iterate through the movie names in bond_data df
    movie_list.append(row['movie'])

    # iterate through bond_data df based on the relevant colums required for the machine learning model to be ran
    X_predict_scaled=X_scaler.transform(row[['year','budget_adj_2020','film_length', 'avg_imdb_rating','avg_rtn_tom_rating','kills_bond','actor id']].values.reshape(-1,7))
    expected_revenue=y_scaler.inverse_transform(model.predict(X_predict_scaled).reshape(-1,1))[0][0]
    predicted_revenue.append(expected_revenue)

In [48]:
df_prediction_tree_reg=pd.DataFrame(
    {"movie":movie_list,
    "predicted_revenue":predicted_revenue}
    )

df_prediction_tree_reg

,movie,predicted_revenue
0,Dr. No,5.257852e+05
1,From Russia with Love,6.873300e+05
2,Goldfinger,1.074015e+06
3,Thunderball,5.727276e+05
4,You Only Live Twice,8.906916e+05
5,On Her Majesty's Secret Service,5.956040e+05
6,Diamonds Are Forever,7.635062e+05
7,Live and Let Die,9.714158e+05
8,The Man with the Golden Gun,5.277314e+05
9,The Spy Who Loved Me,8.155431e+05


In [50]:
df_prediction_tree_reg['predicted_revenue'].round()

0      525785.0
1      687330.0
2     1074015.0
3      572728.0
4      890692.0
5      595604.0
6      763506.0
7      971416.0
8      527731.0
9      815543.0
10     772169.0
11     572728.0
12     501824.0
13     501824.0
14     572728.0
15     335720.0
16     623446.0
17     772169.0
18     335720.0
19     640035.0
20     623446.0
21     732581.0
22    1287090.0
23     992665.0
24    1287090.0
Name: predicted_revenue, dtype: float64

In [51]:
# create classes
Base = declarative_base()

class predicted_revenue(Base):
    __tablename__ = 'bond_predicted_revenue_tree_reg'
    id = Column(Integer, primary_key=True)
    movie = Column(String(255))
    predicted_revenue = Column(Float)

In [52]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [53]:
df_prediction_tree_reg.to_sql(name="bond_predicted_revenue_tree_reg", con=engine, if_exists='append', index=False)

In [59]:
feature_names = ['year','budget_adj_2020','film_length', 'avg_imdb_rating','avg_rtn_tom_rating', 'kills_bond','actor id']
sorted(zip(model.feature_importances_, feature_names), reverse=True)

[(0.5139985602799773, 'avg_rtn_tom_rating'),
 (0.2835408398753189, 'film_length'),
 (0.12834257073302155, 'kills_bond'),
 (0.04624423087101267, 'budget_adj_2020'),
 (0.014604124058148602, 'actor id'),
 (0.011478136444080649, 'year'),
 (0.00179153773844043, 'avg_imdb_rating')]